## Load necessary libraries

First, we are importing libraries (also known as "packages") that we will use throughout this notebook. In this case, we are using pathlib, numpy, and plotly. If they are not installed, we will install them.

Pathlib provides methods to handle file and directory paths, numpy allows us to handle arrays, and plotly is the graphing library we will use for visualization.

If any of these libraries are not installed, we install them directly from this notebook.

In [ ]:
from pathlib import Path

try:
    import numpy as np
except Exception as e:
    print(e)
    %pip install numpy
    import numpy as np


try:
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
except Exception as e:
    print(e)
    %pip install plotly
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    

## Define variables

Next, we're defining a few things we'll need to use later:

- `path_to_recording`: This is a path to the folder where the mocap data for this recording is stored
- `joint_to_plot`: We need to tell the program which joint we want to visualize. By default, we select the 'nose'.
- `mediapipe_indices`: These are the possible joints that can be visualized.

You can select a different joint from `mediapipe_indices` to view the plot for that - for example, you could replace `joint_to_plot = 'nose'` with `joint_to_plot = 'left_elbow'` to view the trajectory visualization for the left elbow. 

We're also getting the path to two types of data we need - 'center of mass' and '3D body data'. We will be loading and using these datasets later on.

In [ ]:
path_to_recording = None

In [ ]:
#pick a joint from the mediapipe indices list to plot
joint_to_plot = 'nose'

In [ ]:

mediapipe_indices = ['nose',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_ear',
    'right_ear',
    'mouth_left',
    'mouth_right',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_pinky',
    'right_pinky',
    'left_index',
    'right_index',
    'left_thumb',
    'right_thumb',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle',
    'left_heel',
    'right_heel',
    'left_foot_index',
    'right_foot_index']

joint_to_plot_index = mediapipe_indices.index(joint_to_plot)

In [ ]:
path_to_recording = Path(path_to_recording)
path_to_center_of_mass_npy = path_to_recording/'output_data'/'center_of_mass'/'total_body_center_of_mass_xyz.npy'
path_to_freemocap_3d_body_data_npy = path_to_recording/'output_data'/'mediapipe_body_3d_xyz.npy'

freemocap_3d_body_data = np.load(path_to_freemocap_3d_body_data_npy)
total_body_com_data = np.load(path_to_center_of_mass_npy)

freemocap_3d_body_data_to_plot = freemocap_3d_body_data[:,joint_to_plot_index,:]


## Plotting

After loading our data, we are going to create some plots to better visualize it. Specifically, we are plotting the trajectory of the total body center of mass and the trajectory of the chosen joint (`nose` by default, but you can replace that and rerun this notebook to plot a different trajectory).

The first three plots (in column 1) represent the X, Y, and Z trajectories of the total body center of mass. The next three plots (in column 2) represent the X, Y, and Z trajectories of the chosen joint.

Note: The X, Y, and Z values refer to the three dimensions in space.

You can click and drag on the plots below to interact with them and zoom into certain areas. When hovering over the plot, you can see additional options in the top right to pan, zoom, reset, and download the plots. 

In [ ]:

fig = make_subplots(rows=3, cols=2, subplot_titles=('total body center of mass trajectory',f'{joint_to_plot} trajectory'))

fig.add_trace(
    go.Scatter(y = total_body_com_data[:,0]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y = total_body_com_data[:,1]),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(y = total_body_com_data[:,2]),
    row=3, col=1
)

fig.add_trace(
        go.Scatter(y = freemocap_3d_body_data_to_plot[:,0]),
    row=1, col=2
)

fig.add_trace(
        go.Scatter(y = freemocap_3d_body_data_to_plot[:,1]),
    row=2, col=2
)

fig.add_trace(
        go.Scatter(y = freemocap_3d_body_data_to_plot[:,2]),
    row=3, col=2
)

#COM plot axes labels 
fig['layout']['yaxis']['title']='X Axis (mm)'
fig['layout']['yaxis3']['title']='Y Axis (mm)'
fig['layout']['yaxis5']['title']='Z Axis (mm)'
fig['layout']['xaxis5']['title']='Frame #'
fig['layout']['xaxis6']['title']='Frame #'




fig.update_layout(height=600, width=800,showlegend=False)
fig.show()


## 3D Plotting

Finally, we are creating a 3D plot of the skeleton movement, tracking all the joints in the `mediapipe_indices` list over time.  

The 'Play' button at the bottom allows you to watch the motion as if it were a video. Before pressing play, you can manually click and drag the plot around to orient the view of the plot. 

In [ ]:

def calculate_axes_means(skeleton_3d_data):
    mx_skel = np.nanmean(skeleton_3d_data[:,0:33,0])
    my_skel = np.nanmean(skeleton_3d_data[:,0:33,1])
    mz_skel = np.nanmean(skeleton_3d_data[:,0:33,2])

    return mx_skel, my_skel, mz_skel

ax_range = 1500

mx_skel, my_skel, mz_skel = calculate_axes_means(freemocap_3d_body_data)

# Create a list of frames
frames = [go.Frame(data=[go.Scatter3d(
    x=freemocap_3d_body_data[i, :, 0],
    y=freemocap_3d_body_data[i, :, 1],
    z=freemocap_3d_body_data[i, :, 2],
    mode='markers',
    marker=dict(
        size=2,  # Adjust marker size as needed
    )
)], name=str(i)) for i in range(freemocap_3d_body_data.shape[0])]

# Define axis properties
axis = dict(
    showbackground=True,
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",
    zerolinecolor="rgb(255, 255, 255)",
)

# Create a figure
fig = go.Figure(
    data=[go.Scatter3d(
        x=freemocap_3d_body_data[0, :, 0],
        y=freemocap_3d_body_data[0, :, 1],
        z=freemocap_3d_body_data[0, :, 2],
        mode='markers',
        marker=dict(
            size=2,  # Adjust marker size as needed
        )
    )],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(axis, range=[mx_skel-ax_range, mx_skel+ax_range]), # Adjust range as needed
            yaxis=dict(axis, range=[my_skel-ax_range, my_skel+ax_range]), # Adjust range as needed
            zaxis=dict(axis, range=[mz_skel-ax_range, mz_skel+ax_range]),  # Adjust range as needed
            aspectmode='cube'
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[dict(
                label='Play',
                method='animate',
                args=[None, {"frame": {"duration": 30}}]
            )]
        )]
    ),
    frames=frames
)

fig.show()